In [1]:
#se importan las librerias para trabajar
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
#Se define una función para realizar el proceso
def productRequest(url):
    
    #Se otbiene el contenido de la URL
    resWeb = requests.get(url)
    web = resWeb.content
    
    #Se aplica la libreria BeautifulSoup para poder realizar busquedas de elementos 
    soupAperitivos = BeautifulSoup(web)
    
    #Se busca el elemento que cotiene los productos que queremos buscar.
    columnaProductos = soupAperitivos.find("div", {"class":"column2"}) #Acá se podría agregar un if en caso de que no existan productos para prevenir posibles errores
     
    #Se crea una lista con todos los productos que aparecen en la página que estamos visualizando.
    listaProductos = columnaProductos.findAll("li", {"class":"product"})
    
    #Creamos el elemento en donde se almacenaran los resultados y que pondremos en el return de la función
    productosProcesados = []

    #Se recorre la lista para cada producto
    for producto in listaProductos:
        a = producto.find("a", {"class":"product-info"}) 
        nombre = a.h6.text
        precio = a.find('span', class_='price').text
        url = a["href"]
        categoria = a.find('span', {'data-content_category': True})['data-content_category']
        #Para cada producto se crea un diccionario para agregar la información correspondiente sobre el mismo
        productoDict = {"Producto": nombre,
                    "Precio": precio,
                    "url": url,
                    "Categoria":categoria}
        
        #Se agrega el producto a la lista de productos.
        productosProcesados.append(productoDict)
        
    return productosProcesados

In [3]:
#Ahora creamos una función que nos diga cuántas páginas tiene una determinada categoria
def totalPages(url):
    try:
        response = requests.get(url , headers=headers)
        content = BeautifulSoup(response.content)
        pages = [x.text for x in content.findAll("a","page-numbers") if "next" not in x.attrs['class']]
        lastPage = int(pages[-1])
    except:
        lastPage=1
    return lastPage

# Scraping
- Realizamos el proceso de scraping. Partimos de las categorias y vamos adquiriendo los productos

In [4]:
#Partimos de la URL productos general.
url = "https://laperlawines.com/product-category/productos/"

In [5]:
#Agreamos el el header identificador de "user-agent" al momento de hacer requests
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'}
response = requests.get(url , headers=headers)
content = BeautifulSoup(response.content)

In [6]:
#Creamos una lista con las categorias para poder luego ingresar a las mismas.
categorias= content.findAll("li","cat-item")
categorias = [x.find("a").get("href") for x in categorias] 
categorias = [x for x in categorias if "https://laperlawines.com/product-category/productos/"!=x] 
categorias

['https://laperlawines.com/product-category/productos/destilados/aperitivos/',
 'https://laperlawines.com/product-category/bazar/',
 'https://laperlawines.com/product-category/productos/cervezas/',
 'https://laperlawines.com/product-category/productos/destilados/',
 'https://laperlawines.com/product-category/productos/extras/',
 'https://laperlawines.com/product-category/productos/destilados/gin/',
 'https://laperlawines.com/product-category/new-in/',
 'https://laperlawines.com/product-category/ofertas/',
 'https://laperlawines.com/product-category/otros/',
 'https://laperlawines.com/product-category/productos/destilados/otros-destilados/',
 'https://laperlawines.com/product-category/regalos-navidenos/',
 'https://laperlawines.com/product-category/productos/sin-alcohol/',
 'https://laperlawines.com/product-category/productos/vinos-y-espumantes/',
 'https://laperlawines.com/product-category/productos/destilados/vodka/',
 'https://laperlawines.com/product-category/productos/destilados/wh

In [7]:
resultados=[]

#Hacemos un loop que corra para todas las categorias
for urlCategoria in categorias:
    
    #Consultamos cuál es la cantidad de páginas para la categoria en cuestión
    lastPage = totalPages(urlCategoria)
    
#     print(urlCategoria)
    
    #Acá al range hay que agregarle un +1 a la cota superior, porque por definición no incluye el último número y nos estariamos perdiendo una página
    for page in range(1,lastPage+1):
        
        #Construimos la url que vamos a buscar. A la categoría se le agrega el número de página
        urlProducto = f"{urlCategoria}page/{page}"
        
#         print(urlProducto)
        
        #Realizamos el scraping de la página del producto, lo almacenamos en la variable newInput y luego lo agregamos al elemento resultados, que concatenará toda la información que queremos.
        newInput = productRequest(urlProducto)
        resultados.extend(newInput)

In [8]:
#Guardamos el resultado en un dataframe de pandas y luego lo expotamos a excel.
resultadosDF = pd.DataFrame(resultados)
resultadosDF.to_excel("LaPerlaWines.xlsx",index=False)
resultadosDF

,Producto,Precio,url,Categoria
0,Aperol,"$2,790.00",https://laperlawines.com/product/aperol/,PRODUCTOS
1,Branca Menta x450,Consultar precio,https://laperlawines.com/product/branca-menta/,PRODUCTOS
2,Branca Menta x750,Consultar precio,https://laperlawines.com/product/branca-menta-...,PRODUCTOS
3,Campari 0.05,Consultar precio,https://laperlawines.com/product/campari/,PRODUCTOS
4,Campari 1L,Consultar precio,https://laperlawines.com/product/campari-1l/,PRODUCTOS
...,...,...,...,...
1137,WHITE & MACKAY WHISKY,Consultar precio,https://laperlawines.com/product/white-mackay-...,WHISKY
1138,WILD TURKEY 101 WHISKY,Consultar precio,https://laperlawines.com/product/wild-turkey-1...,WHISKY
1139,WILD TURKEY WHISKY 40%,Consultar precio,https://laperlawines.com/product/wild-turkey-1...,WHISKY
1140,WILD TURKEY WHISKY AMERICAN HONEY,Consultar precio,https://laperlawines.com/product/wild-turkey-w...,WHISKY
